# 셀프 쿼리(Self-querying)

사용자가 자연어로 입력한 쿼리에서 LLM이 직접 메타데이터 필터 조건을 추론하고, 해당 조건을 기반으로 검색 쿼리를 구성하는 리트리버

예:
"2023년 이후에 발행된 기술 백서 중, AI와 관련된 것"
→ LLM이 자동으로 "published_year >= 2023" 및 "category == AI" 조건을 추출하여 검색.

[ 장점 ]
1. 사용자 친화적인 자연어 질의 가능
사용자는 쿼리 언어나 필터 조건을 명시할 필요 없이 자연어로만 질문하면 됨.

복잡한 필터 조건도 LLM이 추론해서 처리.
장점: UX 개선, 비전문가도 고급 검색 가능.


2. 정확한 메타데이터 기반 검색
단순 임베딩 유사도 검색이 아닌, 정형화된 메타데이터 필터링을 추가할 수 있음.

예: 날짜, 출처, 저자, 문서 유형 등.
장점: 정확도 향상 + 정밀 검색 가능.

3. 벡터 검색과 메타검색의 결합
벡터 검색(의미 기반)과 메타데이터 검색(속성 기반)을 하이브리드로 사용 가능.

예: "2022년 이후 AI 기술 중 '프라이버시 보호'를 다룬 문서"
장점: 의미적 정확성과 필터 기반 제약 조건을 동시에 만족.

4. 복잡한 검색 조건도 자동 처리
날짜 범위, 범주, 특정 키워드 조합 등의 복잡한 검색 조건을 자동 생성 가능.
장점: 검색 쿼리의 복잡도를 LLM이 해결.

[ 단점 ]
1. LLM 추론 정확도에 의존
잘못된 필터 조건을 추론할 수 있음.
예: "2022년 이후"를 "2021"로 인식하는 오류 등.
단점: 잘못된 결과를 줄 위험 존재.

2. 비용 증가 (LLM 호출 필요)
쿼리마다 LLM이 실행되어야 하므로, API 호출 비용 및 시간 증가.
단점: 속도 및 비용 부담.

3. 스키마와 필드 설명 필요
LLM이 메타데이터를 잘 추론하려면, **문서 필드 설명(schema description)**이 명확히 주어져야 함.
단점: 세팅이 까다롭고 사전 정의된 스키마가 필요함.

4. 디버깅 어려움
필터가 LLM 내부에서 생성되므로, 조건이 어떻게 해석되었는지 추적이 어려움.
결과가 이상할 경우 원인을 파악하기 힘듦.
단점: 투명성 및 디버깅 어려움.




`SelfQueryRetriever` 는 자체적으로 질문을 생성하고 해결할 수 있는 기능을 갖춘 검색 도구입니다. 

이는 사용자가 제공한 자연어 질의를 바탕으로, `query-constructing` LLM chain을 사용해 구조화된 질의를 만듭니다. 그 후, 이 구조화된 질의를 기본 벡터 데이터 저장소(VectorStore)에 적용하여 검색을 수행합니다.

이 과정을 통해, `SelfQueryRetriever` 는 단순히 사용자의 입력 질의를 저장된 문서의 내용과 의미적으로 비교하는 것을 넘어서, 사용자의 질의에서 문서의 메타데이터에 대한 **필터를 추출** 하고, 이 필터를 실행하여 관련된 문서를 찾을 수 있습니다. 

[참고]

- LangChain 이 지원하는 셀프 쿼리 검색기(Self-query Retriever) 목록은 [여기](https://python.langchain.com/docs/integrations/retrievers/self_query) 에서 확인해 주시기 바랍니다.


NC 상품의 설명과 메타데이터를 기반으로 유사도 검색이 가능한 벡터 저장소를 구축합니다.

In [ ]:
from dotenv import load_dotenv                          # API 키를 환경변수로 관리하기 위한 설정 파일
from langchain_chroma import Chroma                     # Chroma: 문서를 벡터로 변환하여 저장하고 검색할 수 있는 벡터 데이터베이스 인터페이스
from langchain_core.documents import Document           # Document: LangChain에서 사용하는 표준 문서 객체로, 텍스트 콘텐츠와 메타데이터를 포함
from langchain_openai import OpenAIEmbeddings           # OpenAIEmbeddings: OpenAI의 임베딩 API를 사용하여 텍스트 데이터를 벡터로 변환

# 네이버클라우드드 상품의 설명과 메타데이터 생성

docs = [
    Document(
        page_content="보안이 강화된 네트워크 분리 환경으로, 외부 인터넷과의 접속을 차단하고 내부적으로만 운영 가능한 고보안 환경을 제공합니다.",
        metadata={"year": 2024, "category": "보안", "user_rating": 4.7},
    ),
    Document(
        page_content="서버, 데이터베이스, 파일 등을 자동 백업하고 복구할 수 있습니다.",
        metadata={"year": 2023, "category": "백업", "user_rating": 4.5},
    ),
    Document(
        page_content="웹 애플리케이션에 대한 공격을 차단하는 보안 장비로, SQL Injection, XSS 등 OWASP Top10 위협으로부터 보호합니다.",
        metadata={"year": 2023, "category": "보안", "user_rating": 4.8},
    ),
    Document(
        page_content="네트워크 기반의 공유 저장소로, 여러 서버에서 동시에 데이터를 저장하거나 읽을 수 있게 해줍니다.",
        metadata={"year": 2023, "category": "스토리지", "user_rating": 4.6},
    ),
    Document(
        page_content="이미지, 동영상, 로그, 백업 파일 등 비정형 데이터를 저장하고 관리할 수 있습니다.",
        metadata={"year": 2024, "category": "스토리지", "user_rating": 4.4},
    ),
    Document(
        page_content="이미지나 문서에서 텍스트를 추출하고, 인식한 정보를 구조화하는 AI 기반의 문자인식 서비스입니다.",
        metadata={"year": 2024, "category": "이미지", "user_rating": 4.9},
    ),
]

# 벡터 저장소 생성
vectorstore = Chroma.from_documents(
    docs, OpenAIEmbeddings(model="text-embedding-3-small")
)

results = vectorstore._collection.get(include=['documents', 'metadatas'])

print("문서 내용 :")
for doc in results['documents']:
    print(doc)

print("메타데이터 :")
for meta in results['metadatas']:
    print(meta)

문서 내용들:
보안이 강화된 네트워크 분리 환경으로, 외부 인터넷과의 접속을 차단하고 내부적으로만 운영 가능한 고보안 환경을 제공합니다.
서버, 데이터베이스, 파일 등을 자동 백업하고 복구할 수 있습니다.
웹 애플리케이션에 대한 공격을 차단하는 보안 장비로, SQL Injection, XSS 등 OWASP Top10 위협으로부터 보호합니다.
네트워크 기반의 공유 저장소로, 여러 서버에서 동시에 데이터를 저장하거나 읽을 수 있게 해줍니다.
이미지, 동영상, 로그, 백업 파일 등 비정형 데이터를 저장하고 관리할 수 있습니다.
이미지나 문서에서 텍스트를 추출하고, 인식한 정보를 구조화하는 AI 기반의 문자인식 서비스입니다.
메타데이터들:
{'category': '보안', 'user_rating': 4.7, 'year': 2024}
{'category': '백업', 'user_rating': 4.5, 'year': 2023}
{'category': '보안', 'user_rating': 4.8, 'year': 2023}
{'category': '스토리지', 'user_rating': 4.6, 'year': 2023}
{'category': '스토리지', 'user_rating': 4.4, 'year': 2024}
{'category': '이미지', 'user_rating': 4.9, 'year': 2024}


## SelfQueryRetriever

이제 retriever를 인스턴스화할 수 있습니다. 이를 위해서는 문서가 지원하는 **메타데이터 필드** 와 문서 내용에 대한 **간단한 설명을 미리 제공** 해야 합니다.


`AttributeInfo` 클래스를 사용하여 화장품 메타데이터 필드에 대한 정보를 정의합니다.

- 카테고리(`category`): 문자열 타입, 상품의 카테고리를 나타내며 ['백업', '스토리지토리지', '보안', '이미지'] 중 하나의 값을 가집니다.
- 연도(`year`): 정수 타입, 상품이 출시된 연도를 나타냅니다.
- 사용자 평점(`user_rating`): 실수 타입, 1-5 범위의 사용자 평점을 나타냅니다.

In [4]:
from langchain.chains.query_constructor.base import AttributeInfo # `AttributeInfo` 클래스를 사용하여 화장품 메타데이터 필드에 대한 정보를 정의


# 메타데이터 필드 정보 생성

metadata_field_info = []

metadata_field_info = [
    AttributeInfo(
        name="category",
        description="The category of the NaverCloud product. One of ['백업', '스토리지', '보안', '이미지']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the NaverCloud product was released",
        type="integer",
    ),
    AttributeInfo(
        name="user_rating",
        description="A user rating for the NaverCloud product, ranging from 1 to 5",
        type="float",
    ),
]

`SelfQueryRetriever.from_llm()` 메서드를 사용하여 `retriever` 객체를 생성합니다.

- `llm`: 언어 모델
- `vectorstore`: 벡터 저장소
- `document_contents`: 문서들의 내용 설명
- `metadata_field_info`: 메타데이터 필드 정보


In [5]:
from langchain.retrievers.self_query.base import SelfQueryRetriever  # SelfQueryRetriever 클래스를 임포트: 사용자의 자연어 쿼리를 분석해 벡터 검색에 활용하는 검색기
from langchain_openai import ChatOpenAI                              # OpenAI ChatGPT 모델 연동을 위한 클래스 임포트

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# SelfQueryRetriever 생성: LLM과 벡터 저장소를 결합하여 문서 검색 기능을 수행하는 검색기 생성
retriever = SelfQueryRetriever.from_llm(
    llm=llm,                                                     # 쿼리 분석에 사용할 LLM
    vectorstore=vectorstore,                                     # 유사도 검색을 위한 벡터 저장소
    document_contents="Brief summary of a NaverCloud product",   # 문서 내용 설명 (메타데이터 필드 해석에 도움)
    metadata_field_info=metadata_field_info,                     # 문서 메타데이터 필드 정보 (필드 이름, 타입 등)
)

## Query 테스트

필터를 걸 수 있는 질의를 입력하여 검색을 수행합니다.

In [6]:
# Self-query 검색
retriever.invoke("평점이 4.8 이상인 제품을 추천해주세요.")

[Document(id='2bb4dc5e-a613-43cb-8f89-9bab616295f0', metadata={'category': '보안', 'user_rating': 4.8, 'year': 2023}, page_content='웹 애플리케이션에 대한 공격을 차단하는 보안 장비로, SQL Injection, XSS 등 OWASP Top10 위협으로부터 보호합니다.'),
 Document(id='f03e1723-e4fb-47fd-9d2e-db2c2a48d146', metadata={'category': '이미지', 'user_rating': 4.9, 'year': 2024}, page_content='이미지나 문서에서 텍스트를 추출하고, 인식한 정보를 구조화하는 AI 기반의 문자인식 서비스입니다.')]

In [7]:
# Self-query 검색
retriever.invoke("2023년에 출시된 상품을 추천해주세요")

[Document(id='2bb4dc5e-a613-43cb-8f89-9bab616295f0', metadata={'category': '보안', 'user_rating': 4.8, 'year': 2023}, page_content='웹 애플리케이션에 대한 공격을 차단하는 보안 장비로, SQL Injection, XSS 등 OWASP Top10 위협으로부터 보호합니다.'),
 Document(id='ff6260e8-9fb1-4f8f-a23c-c4cf4009f61c', metadata={'category': '백업', 'user_rating': 4.5, 'year': 2023}, page_content='서버, 데이터베이스, 파일 등을 자동 백업하고 복구할 수 있습니다.'),
 Document(id='68624227-2a58-4443-8389-611499f89ffb', metadata={'category': '스토리지', 'user_rating': 4.6, 'year': 2023}, page_content='네트워크 기반의 공유 저장소로, 여러 서버에서 동시에 데이터를 저장하거나 읽을 수 있게 해줍니다.')]

In [8]:
# Self-query 검색
retriever.invoke("카테고리가 스토리지지인 상품을 추천해주세요")

[Document(id='685a1588-8cc3-4a98-8fa3-be30deb4d355', metadata={'category': '스토리지', 'user_rating': 4.4, 'year': 2024}, page_content='이미지, 동영상, 로그, 백업 파일 등 비정형 데이터를 저장하고 관리할 수 있습니다.'),
 Document(id='68624227-2a58-4443-8389-611499f89ffb', metadata={'category': '스토리지', 'user_rating': 4.6, 'year': 2023}, page_content='네트워크 기반의 공유 저장소로, 여러 서버에서 동시에 데이터를 저장하거나 읽을 수 있게 해줍니다.')]

복합 필터를 사용하여 검색을 수행할 수 있습니다.

In [9]:
# Self-query 검색
retriever.invoke(
    "카테고리가 보안인 상품 중에서 평점이 4.5 이상인 상품을 추천해주세요"
)

[Document(id='2bb4dc5e-a613-43cb-8f89-9bab616295f0', metadata={'category': '보안', 'user_rating': 4.8, 'year': 2023}, page_content='웹 애플리케이션에 대한 공격을 차단하는 보안 장비로, SQL Injection, XSS 등 OWASP Top10 위협으로부터 보호합니다.'),
 Document(id='63a5d87a-7daf-4e4d-b4f1-d1a92e8817bf', metadata={'category': '보안', 'user_rating': 4.7, 'year': 2024}, page_content='보안이 강화된 네트워크 분리 환경으로, 외부 인터넷과의 접속을 차단하고 내부적으로만 운영 가능한 고보안 환경을 제공합니다.')]

`k`는 가져올 문서의 수를 의미합니다.

`SelfQueryRetriever`를 사용하여 `k`를 지정할 수도 있습니다. 이는 생성자에 `enable_limit=True`를 전달하여 수행할 수 있습니다.


In [12]:
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    document_contents="Brief summary of a cosmetic product",
    metadata_field_info=metadata_field_info,
    enable_limit=True,  # 검색 결과 개수 제한 기능 활성화 (True면 아래 search_kwargs의 k값 적용)
    search_kwargs={"k": 2},  # k 의 값을 2로 지정하여 검색 결과를 2개로 제한합니다.
)

2023년도 출시된 상품은 3개가 있지만 "k" 값을 2로 지정하여 2개만 반환하도록 합니다.

In [13]:
# Self-query 검색
retriever.invoke("2023년에 출시된 상품을 추천해주세요")

[Document(id='2bb4dc5e-a613-43cb-8f89-9bab616295f0', metadata={'category': '보안', 'user_rating': 4.8, 'year': 2023}, page_content='웹 애플리케이션에 대한 공격을 차단하는 보안 장비로, SQL Injection, XSS 등 OWASP Top10 위협으로부터 보호합니다.'),
 Document(id='ff6260e8-9fb1-4f8f-a23c-c4cf4009f61c', metadata={'category': '백업', 'user_rating': 4.5, 'year': 2023}, page_content='서버, 데이터베이스, 파일 등을 자동 백업하고 복구할 수 있습니다.')]

하지만 코드로 명시적으로 `search_kwargs`를 지정하지 않고 query 에서 `1개, 2개` 등의 숫자를 사용하여 검색 결과를 제한할 수 있습니다.


In [14]:
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    document_contents="Brief summary of a NaverCloud product",
    metadata_field_info=metadata_field_info,
    enable_limit=True,  # 검색 결과 제한 기능을 활성화합니다.
)

# Self-query 검색
retriever.invoke("2023년에 출시된 상품 1개를 추천해주세요")

[Document(id='2bb4dc5e-a613-43cb-8f89-9bab616295f0', metadata={'category': '보안', 'user_rating': 4.8, 'year': 2023}, page_content='웹 애플리케이션에 대한 공격을 차단하는 보안 장비로, SQL Injection, XSS 등 OWASP Top10 위협으로부터 보호합니다.')]

In [15]:
# Self-query 검색
retriever.invoke("2023년에 출시된 상품 2개를 추천해주세요")

[Document(id='2bb4dc5e-a613-43cb-8f89-9bab616295f0', metadata={'category': '보안', 'user_rating': 4.8, 'year': 2023}, page_content='웹 애플리케이션에 대한 공격을 차단하는 보안 장비로, SQL Injection, XSS 등 OWASP Top10 위협으로부터 보호합니다.'),
 Document(id='ff6260e8-9fb1-4f8f-a23c-c4cf4009f61c', metadata={'category': '백업', 'user_rating': 4.5, 'year': 2023}, page_content='서버, 데이터베이스, 파일 등을 자동 백업하고 복구할 수 있습니다.')]

## 더 깊게 들어가기

내부에서 어떤 일이 일어나는지 확인하고 더 많은 사용자 정의 제어를 하기 위해, 우리는 retriever를 처음부터 재구성할 수 있습니다.

이 과정은 `query-construction chain` 을 생성하는 것부터 시작합니다.

- [참고 튜토리얼](https://github.com/langchain-ai/langchain/blob/master/cookbook/self_query_hotel_search.ipynb) 

### `query_constructor` chain 생성

구조화된 쿼리를 생성하는 `query_constructor` chain 을 생성합니다.

`get_query_constructor_prompt` 함수를 사용하여 쿼리 생성기 프롬프트를 가져옵니다.


In [ ]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,                        # StructuredQueryOutputParser: LLM의 출력 결과를 StructuredQuery 객체로 파싱하는 데 사용
    get_query_constructor_prompt,                       # get_query_constructor_prompt: 쿼리 생성을 위한 시스템 프롬프트 템플릿을 반환
)

# 문서 내용 설명과 메타데이터 필드 정보를 사용하여 쿼리 생성기 프롬프트를 가져옵니다.
prompt = get_query_constructor_prompt(
    "Brief summary of a NaverCloud product",  # 문서 내용 설명
    metadata_field_info,  # 메타데이터 필드 정보
)

# LLM이 생성한 텍스트를 StructuredQuery 형식으로 변환하기 위한 파서 생성
output_parser = StructuredQueryOutputParser.from_components()

# query_constructor chain 을 생성
# 쿼리 생성 체인 정의
# 사용자 입력 → prompt → LLM 처리 → 파서 → StructuredQuery 객체 반환

query_constructor = prompt | llm | output_parser

`prompt.format()` 메서드를 사용하여 `query` 매개변수에 "dummy question" 문자열을 전달하고, 그 결과를 출력하여 Prompt 내용을 확인해 보겠습니다.


In [19]:
# prompt 출력
print(prompt.format(query="dummy question"))

Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | or | not

`query_constructor.invoke()` 메서드를 호출하여 주어진 쿼리에 대한 처리를 수행합니다.


In [24]:
query_output = query_constructor.invoke(
    {
        # 쿼리 생성기를 호출하여 주어진 질문에 대한 쿼리를 생성합니다.
        "query": "2023년도에 출시한 상품 중 평점이 4.5 이상인 상품중에서 보안 제품을 추천해주세요"
    }
)

생성된 쿼리를 확인해 보겠습니다.

eq = 같다, gte = 크거나 같다

In [25]:
# 쿼리 출력
query_output.filter.arguments

[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='year', value=2023),
 Comparison(comparator=<Comparator.GTE: 'gte'>, attribute='user_rating', value=4.5),
 Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='category', value='보안')]

Self-query retriever의 핵심 요소는 query constructor입니다. 훌륭한 검색 시스템을 만들기 위해서는 query constructor가 잘 작동하도록 해야 합니다.

이를 위해서는 **프롬프트(Prompt), 프롬프트 내의 예시, 속성 설명 등을 조정** 해야 합니다.

### 구조화된 쿼리 변환기(Structured Query Translator)를 사용하여 구조화된 쿼리로 변환


다음으로 중요한 요소는 structured query translator입니다. 

이는 일반적인 `StructuredQuery` 객체를 사용 중인 vector store의 구문에 맞는 메타데이터 필터로 변환하는 역할을 담당합니다.

In [ ]:
from langchain.retrievers.self_query.chroma import ChromaTranslator    # SelfQueryRetriever 및 ChromaTranslator를 임포트

retriever = SelfQueryRetriever(
    query_constructor=query_constructor,             # 이전에 생성한 query_constructor chain 을 지정
    vectorstore=vectorstore,                         # 벡터 저장소를 지정
    structured_query_translator=ChromaTranslator(),  # 구조화 쿼리를 Chroma 벡터 DB에 맞는 검색 필터 형식으로 번역해주는 변환기
)

`retriever.invoke()` 메서드를 사용하여 주어진 질문에 대한 답변을 생성합니다.


In [28]:
retriever.invoke(
    # 질문
    "2023년도에 출시한 상품 중 평점이 4.5 이상인 상품중에서 보안안 제품을 추천해주세요"
)

[Document(id='2bb4dc5e-a613-43cb-8f89-9bab616295f0', metadata={'category': '보안', 'user_rating': 4.8, 'year': 2023}, page_content='웹 애플리케이션에 대한 공격을 차단하는 보안 장비로, SQL Injection, XSS 등 OWASP Top10 위협으로부터 보호합니다.'),
 Document(id='ff6260e8-9fb1-4f8f-a23c-c4cf4009f61c', metadata={'category': '백업', 'user_rating': 4.5, 'year': 2023}, page_content='서버, 데이터베이스, 파일 등을 자동 백업하고 복구할 수 있습니다.'),
 Document(id='68624227-2a58-4443-8389-611499f89ffb', metadata={'category': '스토리지', 'user_rating': 4.6, 'year': 2023}, page_content='네트워크 기반의 공유 저장소로, 여러 서버에서 동시에 데이터를 저장하거나 읽을 수 있게 해줍니다.')]